In [1]:
# to access functions from root directory
import sys
sys.path.append('/data/ad181/RemoteDir/multilevel_ppo')

In [2]:
%matplotlib notebook
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from tqdm.notebook import trange, tqdm

import gym
from stable_baselines3.ppo import PPO, MlpPolicy
from stable_baselines3.ppo_multi_level import PPO_ML
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env.subproc_vec_multi_level_env import SubprocVecMultiLevelEnv
from stable_baselines3.common.envs.multi_level_ressim_env import MultiLevelRessimEnv
from stable_baselines3.common.logger import configure

from utils.custom_eval_callback import CustomEvalCallback, CustomEvalCallbackParallel
from utils.plot_functions import plot_learning
from utils.env_evaluate_functions import eval_actions

In [3]:
seed=1
case='ppo_2l'
data_dir='./data'
log_dir='./data/'+case

In [4]:
os.makedirs(data_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [5]:
with open('../envs_params/env_data_v1/env_train_dict.pkl', 'rb') as input:
    env_ck_dict = pickle.load(input)

In [6]:
# generate dictionaries for env (env_dict_), n_steps (T_ml) and minibatch (M_ml) for `n_level`s
n_levels=2
fine_level = len(env_ck_dict)
env_dict_ = {}
for i,l in enumerate(range(fine_level-n_levels, fine_level)):
    print(i+1,'->',l+1)
    env_dict_[i+1] = env_ck_dict[l+1]

1 -> 4
2 -> 5


In [7]:
for seed in range(1,4):
    if True: 
        print(f'seed {seed}')
        log_dir = './data/'+case+'/seed_'+str(seed)
        os.makedirs(log_dir, exist_ok=True)
        T = {1:70, 2:5} # n_steps
        N = 50 # number of actors
        M = {1:350, 2:25} # minibatch size
        I = 300 # number of iterations
        K = 20 # number of epochs
        
        log_interval = I/30
        
        fine_level = len(env_dict_)
        
        print('generate callback ...')
        eval_callback = CustomEvalCallback( env_dict_[fine_level], 
                                            best_model_save_path=str(log_dir)+'/best_model', 
                                            n_eval_episodes=1,
                                            log_path=str(log_dir)+'/results_eval', 
                                            eval_freq=log_interval*sum(T.values()) )
        
        print('vectorize environment ...')
        
        # generate PPO_ML parameters for MLMC analysis. 
        env_dict = {}
        n_steps_dict = {}
        batch_size_dict = {}
        for env, level in zip(env_dict_.values(), env_dict_.keys()):
            print(f"vectorize env level {level}")
            env_dict[level] = make_vec_env( MultiLevelRessimEnv, 
                                    n_envs=N, 
                                    seed=seed, 
                                    env_kwargs= {"ressim_params":env.ressim_params, "level":env.level}, 
                                    vec_env_cls=SubprocVecMultiLevelEnv )
            n_steps_dict[level] = T[level]
            batch_size_dict[level] = M[level]
        
        print(env_dict_[level].observation_space)
        print('model definition ..')
        model = PPO_ML(policy=MlpPolicy,
                           env=env_dict,
                           learning_rate = 1e-5,
                           n_steps = n_steps_dict,
                           batch_size = batch_size_dict,
                           n_epochs = K,
                           clip_range = 0.1,
                           ent_coef = 0.001,
                           vf_coef = 0.5,
                           policy_kwargs = dict(net_arch=[150,100,80], log_std_init=-2.9),
                           verbose = 1,
                           seed = seed,
                           target_kl = 0.05,
                           device = "auto")
        # set logger for the model
        new_logger = configure(log_dir)
        model.set_logger(new_logger)
        print('policy learning ..')
        model.learn(total_timesteps=N*sum(T.values())*I, callback=eval_callback)
        model.save(log_dir+'/PPO', exclude=['env_dict'])
        del model
        for level in env_dict.keys():
            env_dict[level].close()


seed 1
generate callback ...
vectorize environment ...
vectorize env level 1
vectorize env level 2
Box(-1.0, 1.0, (96,), float64)
model definition ..
Using cuda device
Logging to ./data/ppo_2l/seed_1
policy learning ..


/data/ad181/RemoteDir/multilevel_ppo/utils/custom_eval_callback.py:146: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_multi_level_env.SubprocVecMultiLevelEnv object at 0x7f22cda25b90> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7f22cda65150>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5        |
|    ep_rew_mean     | 0.578    |
| time/              |          |
|    fps             | 72       |
|    iterations      | 1        |
|    time_elapsed    | 51       |
|    total_timesteps | 3750     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5           |
|    ep_rew_mean          | 0.575       |
| time/                   |             |
|    fps                  | 80          |
|    iterations           | 2           |
|    time_elapsed         | 93          |
|    total_timesteps      | 7500        |
| train/                  |             |
|    approx_kl            | 0.007996517 |
|    clip_fraction        | 0.341       |
|    clip_range           | 0.1         |
|    entropy_loss         | -94.8       |
|    explained_variance   | -4.02       |
|    learning_rate        | 1e

/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):


Eval num_timesteps=37500, episode_reward=0.59 +/- 0.00
Episode length: 5.00 +/- 0.00
New best mean reward!
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 5          |
|    mean_reward          | 0.594      |
| rollout/                |            |
|    ep_len_mean          | 5          |
|    ep_rew_mean          | 0.582      |
| time/                   |            |
|    fps                  | 83         |
|    iterations           | 10         |
|    time_elapsed         | 448        |
|    total_timesteps      | 37500      |
| train/                  |            |
|    approx_kl            | 0.02993982 |
|    clip_fraction        | 0.451      |
|    clip_range           | 0.1        |
|    entropy_loss         | -94.8      |
|    explained_variance   | 0.627      |
|    learning_rate        | 1e-05      |
|    loss                 | 0.0305     |
|    n_updates            | 180        |
|    policy_gradient_loss | 0.00

...

Early stopping at step 12 due to reaching max kl: 0.07
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 5         |
|    ep_rew_mean          | 0.655     |
| time/                   |           |
|    fps                  | 86        |
|    iterations           | 281       |
|    time_elapsed         | 12138     |
|    total_timesteps      | 1053750   |
| train/                  |           |
|    approx_kl            | 0.0406888 |
|    clip_fraction        | 0.436     |
|    clip_range           | 0.1       |
|    entropy_loss         | -95       |
|    explained_variance   | 0.883     |
|    learning_rate        | 1e-05     |
|    loss                 | 0.0425    |
|    n_updates            | 5600      |
|    policy_gradient_loss | 0.00244   |
|    std                  | 0.0549    |
|    value_loss           | 0.00464   |
---------------------------------------
Early stopping at step 9 due to reaching max kl: 0.08
-----------

Early stopping at step 11 due to reaching max kl: 0.03
Eval num_timesteps=1087500, episode_reward=0.67 +/- 0.00
Episode length: 5.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 5           |
|    mean_reward          | 0.665       |
| rollout/                |             |
|    ep_len_mean          | 5           |
|    ep_rew_mean          | 0.654       |
| time/                   |             |
|    fps                  | 86          |
|    iterations           | 290         |
|    time_elapsed         | 12522       |
|    total_timesteps      | 1087500     |
| train/                  |             |
|    approx_kl            | 0.025507905 |
|    clip_fraction        | 0.434       |
|    clip_range           | 0.1         |
|    entropy_loss         | -95         |
|    explained_variance   | 0.888       |
|    learning_rate        | 1e-05       |
|    loss                 | 0.0501      |
|    n_updates    

Early stopping at step 14 due to reaching max kl: 0.03
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5           |
|    ep_rew_mean          | 0.656       |
| time/                   |             |
|    fps                  | 86          |
|    iterations           | 299         |
|    time_elapsed         | 12888       |
|    total_timesteps      | 1121250     |
| train/                  |             |
|    approx_kl            | 0.024203675 |
|    clip_fraction        | 0.497       |
|    clip_range           | 0.1         |
|    entropy_loss         | -95         |
|    explained_variance   | 0.894       |
|    learning_rate        | 1e-05       |
|    loss                 | 0.0538      |
|    n_updates            | 5960        |
|    policy_gradient_loss | 0.00372     |
|    std                  | 0.0549      |
|    value_loss           | 0.00404     |
-----------------------------------------
Early stopping at ste

/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p

/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p